# Creating Factor Graphs with Seed and Running Simulations

This notebook demonstrates how to create factor graphs on the spot with a specified seed and run simulations with them.


In [ ]:

import numpy as np
import random
import matplotlib.pyplot as plt

# Import necessary modules for factor graph creation and simulation
from bp_base.factor_graph import FactorGraph
from bp_base.bp_engine_base import BPEngine
from bp_base.engines_realizations import (
    DampingEngine,
    DampingSCFGEngine, SplitEngine, DampingCROnceEngine
)
from utils.create_factor_graphs_from_config import (
    build_cycle_graph,
    build_random_graph
)
from configs.global_config_mapping import CT_FACTORIES
from utils.path_utils import find_project_root

# Set the project root
PROJECT_ROOT = find_project_root()


## Set a Random Seed for Reproducibility

Setting a seed ensures that the random factor graphs created will be the same each time the notebook is run.


In [ ]:
# Set a seed for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

print(f"Using seed: {SEED}")


## Create Factor Graphs Directly

Instead of loading factor graphs from pickle files, we'll create them directly with our specified seed.


In [ ]:
def create_factor_graph(graph_type="cycle", num_vars=5, domain_size=3, ct_factory="random_int", ct_params=None, density=0.5):
    """
    Create a factor graph directly without going through the config and pickle process.

    Args:
        graph_type (str): Type of graph to create ("cycle" or "random")
        num_vars (int): Number of variables in the graph
        domain_size (int): Size of the domain for each variable
        ct_factory (str): Name of the cost table factory to use
        ct_params (dict): Parameters for the cost table factory
        density (float): Density of the graph (for random graphs)

    Returns:
        FactorGraph: The created factor graph
    """
    if ct_params is None:
        ct_params = {"low": 1, "high": 100}

    # Get the cost table factory function
    ct_factory_fn = CT_FACTORIES[ct_factory]
    if graph_type == "cycle":
        variables,factors,edges= build_cycle_graph(
            num_vars= num_vars,
            domain_size=domain_size,
            ct_factory=ct_factory_fn,
            ct_params=ct_params,
            density=density

        )


    if graph_type == "random":
        variables, factors, edges = build_random_graph(
            num_vars=num_vars,
            domain_size=domain_size,
            ct_factory=ct_factory_fn,
            ct_params=ct_params,
            density=density
        )
    else:
        raise ValueError(f"Unknown graph type: {graph_type}")

    # Create the factor graph
    fg = FactorGraph(variable_li=variables, factor_li=factors, edges=edges)

    return fg
# Create a random factor graph with the specified seed
random_fg = []
for i in range(1):
    random_fg.append(create_factor_graph(
        graph_type="random",
        num_vars=50,
        domain_size=20,
        ct_factory="random_int",
        ct_params={"low":1000, "high":10000},
        density=0.25)
    )



## Run Simulations with Different Engines

Now we'll run simulations with the created factor graphs using different engines, including the DampingEngine to demonstrate that damping works correctly after each step.


In [ ]:

from policies.convergance import ConvergenceConfig


"""
Run simulations with different engines on the given factor graph.

Args:
    factor_graph (List[FactorGraph]): The factor graph to run simulations on
    max_iter (int): Maximum number of iterations to run

Returns:
    dict: Dictionary mapping engine names to their costs over iterations
"""
# Create engines

max_iter = 1000
# Run simulations
results = {}

for graph in random_fg:
    # lb = get_bound(graph)
    # print(lb)
    import pickle
    # import cProfile
    # import pstats
    engine0=BPEngine(factor_graph=pickle.loads(pickle.dumps(graph)),convergence_config=ConvergenceConfig())
    engine1 = SplitEngine(factor_graph=pickle.loads(pickle.dumps(graph)),convergence_config=ConvergenceConfig(),monitor_performance=False,split_factor=0.5)
    engine2 =SplitEngine(factor_graph=pickle.loads(pickle.dumps(graph)),convergence_config=ConvergenceConfig(),monitor_performance=False,split_factor=0.6)
    engine3 = DampingSCFGEngine(factor_graph=pickle.loads(pickle.dumps(graph)),convergence_config=ConvergenceConfig(),monitor_performance=False,split_factor=0.5,damping_factor=0.9)
    engine4= DampingSCFGEngine(factor_graph=pickle.loads(pickle.dumps(graph)),convergence_config=ConvergenceConfig(),monitor_performance=False,damping_factor=0.9,split_factor=0.6)
    engine5=DampingCROnceEngine(factor_graph=pickle.loads(pickle.dumps(graph)),convergence_config=ConvergenceConfig(),monitor_performance=False,damping_factor=0.9,reduction_factor=0.5)
    engines = [engine3,engine5]
    for engine in engines:
        results[engine.name] = []
    # profiler = cProfile.Profile()
    # profiler.enable()
    for engine in engines:
        engine.run(max_iter=max_iter)
        print(f"Completed simulation with {engine.name}. Final cost: {engine.history.costs[-1]}")
        results[engine.name].append(engine.history.costs)


    # profiler.disable()
    # stats = pstats.Stats(profiler)
    # stats.sort_stats('cumulative')
    # stats.print_stats(20)
    print(f"finished one cycle of fg")








# Run simulations on the random factor grap


## Plot the Results

Let's plot the costs over iterations for each engine to visualize the convergence behavior.


In [ ]:
"""
Plot the costs over iterations for each engine.

Args:
    results (dict): Dictionary mapping engine names to their costs over iterations
    title (str): Title for the plot
"""
plt.figure(figsize=(12, 8))

for engine_name, costs in results.items():
    max_len = max_len = max(1000,max(len(c) for c in costs))

    costs = [c + [c[-1]] * (max_len - len(c)) for c in costs]
    plt.plot(np.average(np.array(costs), axis=0), label=engine_name)
    # plt.plot(np.arange(0, max_len, 8), np.array([costs[0][int(i/8)*8] for i in range(0, max_len,8)]),linestyle='',marker='o')
plt.plot(label ="Lower Bound",linestyle='--')
plt.title("Costs over Iterations for Random Factor Graph (Density 0.25)")
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.legend()
plt.grid(True)
plt.show()



# Plot results for random factor graph


## Conclusion

This notebook demonstrates how to create factor graphs on the spot with a specified seed and run simulations with them. We've also verified that the damping functionality works correctly after each step.

Key points:
1. Setting a random seed ensures reproducibility of the factor graphs and simulation results.
2. The fixed damping implementation correctly applies damping after each variable computation step.
3. Different engines show different convergence behaviors on the same factor graph.

In [ ]:
print(engine3.convergence_monitor.get_convergence_summary())

In [ ]:
from policies.save import save_simulation_result
save_simulation_result(engine3,find_project_root()/"data/simulation_results2.json")